# ロジスティック回帰

ベルヌーイ分布に従う変数の統計的回帰モデルの一種．主に二値分類で使われる．

---

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
df = pd.read_csv('/Users/takahashikaisei/mydir/local-repository/machine-learning/dataset/breast_cancer_data.csv')
df = df.drop(['id', 'Unnamed: 32'], axis=1)
df = df.rename(columns={'diagnosis': 'target'})
df['target'] = df['target'].replace({'B': 0, 'M': 1})

X = df.drop(["target"], axis = 1)
y = df.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/var/folders/9x/ntmsqgq55glgc5365114c0v80000gn/T/ipykernel_92571/1830256884.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['target'] = df['target'].replace({'B': 0, 'M': 1})


In [3]:
reg = LogisticRegression()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.956140350877193


/Users/takahashikaisei/mydir/local-repository/machine-learning/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
reg.coef_

array([[-1.76820060e+00, -8.24862389e-02,  6.47633921e-02,
         6.50622901e-04,  7.05653532e-02,  3.29800763e-01,
         4.56651842e-01,  1.97287343e-01,  1.05623902e-01,
         2.12972063e-02, -5.60421775e-02, -8.92550480e-01,
        -7.51265170e-02,  7.87659159e-02,  6.72458859e-03,
         6.46133240e-02,  9.06167046e-02,  2.52048646e-02,
         2.50100362e-02,  5.59196964e-03, -1.82891255e+00,
         3.38572135e-01,  1.49282425e-01,  2.80873101e-02,
         1.24792697e-01,  9.86798099e-01,  1.22074325e+00,
         3.62166854e-01,  3.42347995e-01,  9.25470068e-02]])

In [5]:
reg.intercept_

array([-0.34693447])

これと同じ出力結果を目指す．

シグモイド関数(ロジスティック)の式
$$
\sigma = \frac{1}{1-e^{-x}}
$$
このとき$x$に代入するのは単純な線形回帰の式である．

$$
x = X\bm{w}
$$

In [ ]:
class SLogisticRegression:
    def __init__(self):
        self.w = None
    def fit(self, X, y):
        X = np.insert(X, 0, 1, axis=1)


In [7]:
from sklearn.linear_model import LinearRegression

In [8]:
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [10]:
model.coef_

array([-2.17772056e-01,  4.54546867e-03,  2.37398610e-02,  3.17834750e-04,
        8.46891371e-02, -4.22203525e+00,  1.39799728e+00,  2.14183303e+00,
        1.02709200e-01,  3.32616096e-02,  4.34955932e-01, -6.75847233e-03,
       -2.25202577e-02, -9.23217886e-04,  1.58543207e+01,  6.49034090e-02,
       -3.56546799e+00,  1.05679513e+01,  1.69734069e+00, -7.14644016e+00,
        1.95183121e-01,  7.15937520e-03, -2.43505057e-03, -1.01122332e-03,
        5.42856861e-01,  6.71582941e-02,  3.81191215e-01,  4.64309895e-01,
        5.56787546e-01,  4.30348309e+00])

知ってはいたけど単純に線形回帰の式の$w$を求めてシグモイド関数に通して二値分類してるわけではないね．\
内部ではどのようにcoefが求められているのだろう